In [ ]:
//#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution.Gnuplot;
using BoSSS.Solution.Control;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("JOB_NAME");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "Benchmark" : buildname;
//string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
dbname = String.IsNullOrEmpty(dbname)? "dbname" : dbname;
string table_name = String.Concat(buildname, "_", dbname);

In [ ]:
string CurrentDocDir = Directory.GetCurrentDirectory();
string path = String.Concat(CurrentDocDir,@"/",table_name,".json");
var SubTab = TableExtensions.LoadFromFile(path);

Error: System.IO.FileNotFoundException: Could not find file 'b:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\XdgPoisson\run_17-1-2022_Benchmark.json'.
File name: 'b:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\XdgPoisson\run_17-1-2022_Benchmark.json'
   at System.IO.FileStream.ValidateFileHandle(SafeFileHandle fileHandle)
   at System.IO.FileStream.CreateFileOpenHandle(FileMode mode, FileShare share, FileOptions options)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share, Int32 bufferSize, FileOptions options)
   at System.IO.StreamReader.ValidateArgsAndOpenPath(String path, Encoding encoding, Int32 bufferSize)
   at System.IO.StreamReader..ctor(String path, Encoding encoding, Boolean detectEncodingFromByteOrderMarks)
   at System.IO.File.InternalReadAllText(String path, Encoding encoding)
   at System.IO.File.ReadAllText(String path)
   at BoSSS.Application.BoSSSpad.TableExtensions.LoadFromFile(String fil

In [ ]:
//var SubTab = TableExtensions.LoadFromFile(@"B:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\XdgPoisson\data_weak\Xdg_weakscaling_s_2020-9-9.json");

In [ ]:
// downward compatibility ... new : old
string LSolverCode = SubTab.Columns.Contains("LinearSolverCode")? "LinearSolverCode" : "LinearSolver.SolverCode";
string MatrixAssembly = SubTab.Columns.Contains("MatrixAssembly_time")? "MatrixAssembly_time" : "XdgMatrixAssembly";
string AggInittime = SubTab.Columns.Contains("AggregationBaseInit_time")? "AggregationBaseInit_time" : "Aggregation_basis_init_time";
string SolverInit_time = SubTab.Columns.Contains("SolverInit_time")? "SolverInit_time" : "Solver_Init_time";
string SolverRun_time = SubTab.Columns.Contains("SolverRun_time")? "SolverRun_time" : "Solver_Run_time";

In [ ]:
var tab                        = SubTab.ExtractColumns(LSolverCode, "DOFs","DGdegree:u","maxSolRunT","ProjectName","SessionName","NoIter");
System.Data.DataView dv        = tab.DefaultView;
dv.Sort                        = LSolverCode+",DOFs";
System.Data.DataTable sortedDT = dv.ToTable();
sortedDT.Print();

    LinearSolver.SolverCode DOFs    DGdegree:u maxSolRunT         ProjectName           SessionName                          NoIter 
0:  2                       43840   2          40.9736952         Xdg_weakscaling       J4096_k2_classic_pardiso             1      
1:  2                       84320   2          65.7982942         Xdg_weakscaling       J8000_k2_classic_pardiso             1      
2:  2                       183200  2          170.00860459999998 Xdg_weakscaling       J17576_k2_classic_pardiso            1      
3:  2                       339040  2          527.6745040999999  Xdg_weakscaling       J32768_k2_classic_pardiso            1      
4:  2                       660160  2          2208.329985        Xdg_weakscaling       J64000_k2_classic_pardiso            1      
5:  41                      43840   2          183.4011587        Xdg_weakscaling       J4096_k2_exp_Kcycle_schwarz          14     
6:  41                      84320   2          515.5230200999999  Xdg

In [ ]:
 Func<int, int, PlotRowSelectorEx> Solver_DG_Proj_Ex = delegate (int DG, int Solver){
      return delegate (int iSweep, int iTabRow, 
            IDictionary<string, object> Row, 
            out string Nmn, 
            out PlotFormat Fmt,
            out double xValue, out double yValue) { 
      // - - - - - - - - - - - - - - - - - - - - - - - - 
      // PlotRowSelector:
      // selects, which table row goes to which graph,
      // and the respective color  
      // - - - - - - - - - - - - - - - - - - - - - - - - 
      int k           = Convert.ToInt32(Row["DGdegree:u"]);
      int solver_name = Convert.ToInt32(Row[LSolverCode]);
    
      if(k != DG || solver_name != Solver) {
          // degree does not match -> not in this plot
          Nmn    = null;
          Fmt    = null;
          xValue = 0;
          yValue = 0;
          return;
      }
      
      double mtxAssi  = Convert.ToDouble(Row[MatrixAssembly]);
      double aggBinit = Convert.ToDouble(Row[AggInittime]);
      double slvInit  = Convert.ToDouble(Row[SolverInit_time]);
      double slvRun   = Convert.ToDouble(Row[SolverRun_time]);
      double cores    = Convert.ToDouble(Row["NoOfCores"]);
      
      if(aggBinit > 700)
      Console.Write("aggBinit/slvRun = " + aggBinit + "/" + slvRun);
      
      Fmt           = new PlotFormat();
      Fmt.PointSize = 0.5;
      Fmt.Style     = Styles.LinesPoints;
      Fmt.LineColor = LineColors.Black;
      Fmt.LineWidth = 3;
      
      xValue = cores;
      
      switch(iSweep) {
          case 0:
          yValue       = slvRun;
          Nmn          = "Slv Iter";
          Fmt.DashType = DashTypes.Dotted;
          Fmt.PointType = PointTypes.OpenCircle;
          break;
          
          case 1:
          yValue        = slvInit;
          Nmn           = "Slv Init";
          Fmt.DashType  = DashTypes.Dashed;
          Fmt.PointType = PointTypes.OpenBox;
          break;
          
          case 2:
          yValue        = aggBinit;
          Nmn           = "Agg Init";
          Fmt.DashType  = DashTypes.DotDashed;
          Fmt.PointType = PointTypes.OpenDiamond;
          break;
          
          case 3:
          yValue        = mtxAssi;
          Nmn           = "Mtx ass";
          Fmt.DashType  = DashTypes.DotDotDashed;
          Fmt.PointType = PointTypes.OpenUpperTriangle;
          break;
          
          default:
          throw new NotImplementedException();
      }
      
      if(aggBinit > 700)
      Console.WriteLine("\t + case: {0} + yValue = {1}", iSweep, yValue);
  };
};

In [ ]:
SubTab.GetColumnNames().Where(s=>s.Contains("Run"))

index,value
0,minSolRunT
1,maxSolRunT
2,Solver_Run_time


In [ ]:
var Row = SubTab.Rows[0];

Convert.ToDouble(Row[MatrixAssembly])

22.613752299999998

In [ ]:
// =========
// paramterz
// =========
int Solver = 41;
int DG     = 2;
int xMin = 4;
int xMax = 512;
// =============================
// print profiling of blocktraces
// =============================
// necessary methods, are defined in Export.bws
Plot2Ddata[,] multiplots = new Plot2Ddata[1,1];
multiplots[0,0]=SubTab.ToPlot(Solver_DG_Proj_Ex(DG,Solver),4);
multiplots[0,0].YrangeMin=1;
multiplots[0,0].YrangeMax=10000;
multiplots[0,0].XrangeMin=xMin;
multiplots[0,0].XrangeMax=xMax;
multiplots[0,0].LogX = true;
multiplots[0,0].LogY = true;
multiplots[0,0].bmargin = 3;
multiplots[0,0].Xlabel = "no of cores";
multiplots[0,0].Ylabel = "runtime of proc0";
multiplots.PlotNow()

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 3 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 runtime of proc0 
 
 
 
 
 no of cores 
 
 
 
 
 Slv Iter 
 
 
 
 
 Slv Iter 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Slv Init 
 
 
 Slv Init 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Agg Init 
 
 
 Agg Init 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Mtx ass 
 
 
 Mtx ass

In [ ]:
// ==============
// plot profiling
// ==============
string plotsdirpath = CurrentDocDir+@"\"+"plots";
DirectoryInfo dir   = Directory.CreateDirectory(plotsdirpath);
var plot2print      = multiplots;
var CL              = plot2print.ToGnuplot().PlotCairolatex(xSize:14,ySize:12);
CL.WriteMinimalCompileableExample(
   Path.Combine(plotsdirpath, "Profiling_"+DG+"_"+".tex"),
   "Profiling_"+DG+".tex");

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 
